# Model Evaluation
Compare CF vs SVD vs Hybrid with metrics like RMSE, MAE, Precision@K, Recall@K, and NDCG.

In [ ]:
from collections import defaultdict
from src import collaborative_filtering, data_preprocessing, evaluation, hybrid_model, matrix_factorization

data = data_preprocessing.load_datasets()
books, users, ratings = data_preprocessing.clean_data(**data)  # type: ignore[arg-type]
ratings = data_preprocessing.filter_interactions(ratings)

split = data_preprocessing.split_ratings(ratings)
user_cf = collaborative_filtering.UserBasedCF().fit(split.train)
item_cf = collaborative_filtering.ItemBasedCF().fit(split.train)
svd_model = matrix_factorization.SVDRecommender().fit(split.train)
hybrid = hybrid_model.HybridRecommender(user_cf, item_cf, svd_model)

users_eval = split.test['user_id'].unique()[:100]
rec_dict = {}
truth = defaultdict(list)
for _, row in split.test.iterrows():
    truth[row['user_id']].append(row['isbn'])

for uid in users_eval:
    candidate_items = books['isbn'].tolist()[:500]
    recs = hybrid.recommend(uid, candidate_items, books=books, top_n=10)
    rec_dict[uid] = [rec['isbn'] for rec in recs]

precision = evaluation.precision_at_k(rec_dict, truth, k=10)
recall = evaluation.recall_at_k(rec_dict, truth, k=10)
precision, recall